In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, desc, col
from pyspark.sql.types import IntegerType
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Twitch_Networks.com") \
    .getOrCreate()

22/07/18 18:13:25 WARN Utils: Your hostname, ELISABETH-PERSO resolves to a loopback address: 127.0.1.1; using 10.33.4.168 instead (on interface wlp1s0)
22/07/18 18:13:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/18 18:13:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Import des CSVs

In [21]:
df_top = spark.read.csv("hdfs://127.0.0.1:9000/esgi/data/projets/final/twitchdata-update.csv", 
                           header='true')
df_edges = spark.read.csv("hdfs://127.0.0.1:9000/esgi/data/projets/final/EDGELIST.csv", 
                           header='true')

#### cast string en int

In [23]:
df_top = df_top.withColumn("Followers",col("Followers").cast("integer"))
df_top = df_top.withColumn("Peak viewers",col("Peak viewers").cast("integer"))
df_top = df_top.withColumn("Average viewers",col("Average viewers").cast("integer"))
df_top = df_top.withColumn("Watch time(Minutes)",col("Watch time(Minutes)").cast("integer"))
df_top = df_top.withColumn("Stream time(minutes)",col("Stream time(minutes)").cast("integer"))
df_edges = df_edges.withColumn("Weight",col("Weight").cast("integer"))

+---------------------+-------------------+--------------------+------------+---------------+---------+----------------+------------+---------+------+----------+
|              Channel|Watch time(Minutes)|Stream time(minutes)|Peak viewers|Average viewers|Followers|Followers gained|Views gained|Partnered|Mature|  Language|
+---------------------+-------------------+--------------------+------------+---------------+---------+----------------+------------+---------+------+----------+
|                xQcOW|         6196161750|              215250|      222720|          27716|  3246298|         1734810|    93036735|     True| False|   English|
|             summit1g|         6091677300|              211845|      310998|          25610|  5310163|         1370184|    89705964|     True| False|   English|
|               Gaules|         5644590915|              515280|      387315|          10976|  1767635|         1023779|   102611607|     True|  True|Portuguese|
|             ESL_CSGO|     

StructType([StructField('Source', StringType(), True), StructField('Target', StringType(), True), StructField('Weight', IntegerType(), True)])

### Edges streamers abonnés

In [18]:
df_edges.sort("Source").limit(5).toPandas()

,Source,Target,Weight
0,01bd02,PUBGKorea,922
1,01bd02,ch1ckenkun,2176
2,01bd02,pikra10,3376
3,01bd02,woowakgood,3290
4,01bd02,lol_woolf,1080


### Streamers les plus suivis

In [19]:
df_edges.groupBy("Target").count().sort(desc("count")).toPandas()

,Target,count
0,Indiefoxx,1541
1,loltyler1,1342
2,ESL_CSGO,1121
3,Rainbow6,1074
4,Asmongold,966
...,...,...
2309,ReallyReallyLongAThon,1
2310,rdo1337,1
2311,nwslofficial,1
2312,FenoHS,1


In [26]:
df_edges.sort(desc("Weight")).toPandas()

,Source,Target,Weight
0,RanbooLive,tommyinnit,437725
1,RanbooLive,Tubbo,423393
2,ibai,auronplay,423011
3,Tubbo,tommyinnit,359039
4,RanbooLive,TubboLIVE,333957
...,...,...,...
230293,slwalekoP,HealthyGamer_GG,501
230294,dersin2k,Coreano,501
230295,Horcus,LotharHS,501
230296,sinatraa,Josedeodo,501


### Moyenne des followers des top streamers

# pas utiliser collect -> machine locale

In [32]:
df_top.select(mean("Followers")).toPandas()

,avg(Followers)
0,570054.068


### Streamers par ordre décroissant de followers

In [33]:
df_top.sort(desc("Peak viewers")).toPandas()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
0,Riot Games (riotgames),2674646715,80820,639375,20960,4487489,497678,56855694,True,False,English
1,TheGrefg,1757406750,54855,538444,28887,3795667,3593081,47094362,True,False,Spanish
2,dota2ti,1017577605,6315,483530,147643,663297,121422,16228039,True,False,English
3,shroud,888505170,30240,471281,29612,7744066,833587,30621257,True,False,English
4,dota2ti_ru,812538090,6195,457060,126232,541644,108438,12068376,True,False,Russian
...,...,...,...,...,...,...,...,...,...,...,...
995,voicetv,162510660,514845,1874,314,3660,1328,4001475,True,False,Thai
996,SaltyTeemo,247613265,517980,1803,476,173196,36103,2117741,False,False,English
997,MarketTradersTV,148900110,499215,1357,297,30413,12081,981536,True,False,English
998,runitup247,205746015,486000,1182,422,91608,30397,2424188,True,False,English


### Streamers par ordre décroissant de followers

In [35]:
df_top.sort(desc("Followers")).toPandas()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
0,Tfue,3671000070,123660,285644,29602,8938903,2068424,78998587,True,False,English
1,shroud,888505170,30240,471281,29612,7744066,833587,30621257,True,False,English
2,Myth,1479214575,134760,122552,9396,6726893,1421811,37384058,True,False,English
3,Rubius,2588632635,58275,240096,42948,5751354,3820532,58599449,True,False,Spanish
4,pokimane,964334055,56505,112160,16026,5367605,2085831,45579002,True,False,English
...,...,...,...,...,...,...,...,...,...,...,...
995,칸데르니아 (jmjdoc),131408250,170865,2395,756,30877,9502,4337476,True,False,Korean
996,MarketTradersTV,148900110,499215,1357,297,30413,12081,981536,True,False,English
997,NiteNightKid,178419030,149040,29228,1127,28818,11998,1450208,True,True,English
998,Sinner666,124812240,24765,27996,4530,7102,7101,21322548,False,False,Russian
